# Exploring the implementation of statistical distribution functions

Notebook for exploring and prototyping functions that will be implemented in Mojo. Working in Python to quickly prototype, plot, and explore numerical issues.

In [ ]:
import math

## Error function

The **[error function](https://en.wikipedia.org/wiki/Error_function)** is defined as

$$\mathrm{erf} z = \frac{2}{\sqrt{\pi}} = \int_0^z e^{-t^2} \mathop{dt}$$

which has a [Taylor series](https://en.wikipedia.org/wiki/Error_function#Taylor_series):

$$\mathrm{erf} z = \frac{2}{\sqrt{\pi}} = \sum_{n=0}^\infty \frac{(-1)^n z^{2n+1}}{n!(2n + 1)}$$

Mojo function that we are working with:

```mojo
fn erf_taylor[k: Int = 21](x: Float64) -> Float64:
  var erf_fact: Float64 = 0.0
  var erfz: Float64
  for n in range(k):
    erf_fact += ((-1) ** n * x ** (2 * n + 1)) / (factorial(n) * (2 * n + 1))
  erfz = 2 / sqrt(pi) * erf_fact
  return(erfz)
  ```

  and a Python implementation:

In [ ]:
def erf_taylor(x, k):
  erf_fact = 0.0
  for n in range(k):
    erf_fact += ((-1) ** n * x ** (2 * n + 1)) / (math.factorial(n) * (2 * n + 1))
  erfz = 2 / math.sqrt(math.pi) * erf_fact
  return(erfz)

What's the issue? Above values of about 2.5, the Taylor expansion starts exceeding its bounds of $(-1, 1)$.

In [43]:
xx = [1, 2, 2.5, 2.75, 3.0, 3.25, 3.5, 4, 5, 10, 37]
[erf_taylor(x, 21) for x in xx]

[0.8427007929497148,
 0.9953222688644908,
 0.9996451566471811,
 1.0029016700876547,
 1.1207802033230645,
 4.595628647420463,
 83.79606261718983,
 23274.87610462253,
 276612153.91663253,
 9.333369103156057e+20,
 2.2039065875316448e+44]

How much of a problem is this in practice? The function will be used in the CDF of the normal distribution with the form:

$$\mathrm{erf}\left(\frac{x - \mu}{\sigma \sqrt{2}}\right)$$

So it is standard deviations divided by a factor of $\sqrt{2}$.

In [44]:
[erf_taylor(x/math.sqrt(2), 21) for x in xx]

[0.6826894921370857,
 0.9544997361036428,
 0.9875806693681439,
 0.9940404746870569,
 0.9973002514357023,
 0.9988473925201512,
 0.9995686384297462,
 1.0098520939113542,
 127.91707518264381,
 534744322996248.5,
 1.4640120200395263e+38]

So it still breaks down at not very extreme values (calculating $p \lesssim 3 \times 10^{-5}$).